# Predicting Individual Awards: The Sportsmanship Award

**Project Context**
This notebook addresses **Task (c)** of the project description: "Who won each of the individual awards". Specifically, this analysis focuses on predicting the **Sportsmanship Award**.

**The Logic of Sportsmanship**
Unlike MVP or Scoring titles which are purely performance-based, the Sportsmanship Award honors players who exemplify ethical behavior and integrity. In basketball analytics, this is typically modeled as:
* **High Availability:** Playing significant minutes (essential to the team).
* **Low Conflict:** Committing very few personal fouls or technical fouls relative to playing time.
* **High Performance:** Generally, awards go to recognized, high-quality players (All-Stars), not bench warmers.

**Data Sources**
* **`players_teams`**: Granular statistics (Minutes, PF, DQ, etc.) to model behavior.
* **`awards_players`**: Historical labels for training (Who won previously?).

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ndcg_score

# --- 1. Setup & Data Loading ---
initial_path = '../../data/initial_data/' 
test_path = '../../data/test_data/'

def load_and_combine(filename):
    """
    Loads historical data and appends test data if available.
    Ensures Year 11 is part of the dataframe for feature generation.
    """
    # Load historical data
    df = pd.read_csv(f"{initial_path}{filename}")
    
    # Try to load test data
    try:
        df_test = pd.read_csv(f"{test_path}{filename}")
        # Concatenate
        df = pd.concat([df, df_test], ignore_index=True)
    except FileNotFoundError:
        print(f"Note: No test file found for {filename}")
        pass
    return df

# Load tables
players_teams = load_and_combine('players_teams.csv')
awards = pd.read_csv(f"{initial_path}awards_players.csv")
# We load teams just for context (wins/losses)
teams = load_and_combine('teams.csv')

print(f"Players Stints Loaded: {players_teams.shape}")

### 2. Feature Engineering: Quantifying "Integrity"
To predict "Sportsmanship," we must quantify behavior. We derive a custom metric called the **Discipline Ratio**.

* **`discipline_ratio`**: $\frac{\text{Minutes Played}}{\text{Fouls} + (\text{Disqualifications} \times 50) + 1}$
    * This rewards players who can stay on the floor for long periods without fouling.
    * We heavily penalize **Disqualifications (DQ)** (ejected from game) as they are the antithesis of sportsmanship.

**Handling the "Zero Feature Trap"**
A common issue in forecasting is missing data for the test year (Year 11) or players missing a prior season due to injury. If we rely solely on $t-1$ stats, these players get "0" scores.
* **The Fix**: We calculate **Cumulative Career Averages** using expanding windows. If a player's previous season stats are missing, we fallback to their career baseline.

In [ ]:
# --- Feature Engineering: Integrity & Base Performance ---

# 1. Aggregation (Handle players with multiple stints in one year)
agg_cols = ['GP', 'minutes', 'points', 'rebounds', 'assists', 'steals', 
            'blocks', 'turnovers', 'PF', 'fgMade', 'fgAttempted', 'dq']

# Sum stats to get total season footprint
df_season = players_teams.groupby(['playerID', 'year'])[agg_cols].sum().reset_index()

# 2. Calculate Derived Metrics (Integrity + Efficiency)
# "Lady Byng" Ratio: Reward high minutes with low fouls. Penalty: DQ counts as 50 fouls.
df_season['discipline_ratio'] = df_season['minutes'] / (df_season['PF'] + (df_season['dq'] * 50) + 1)
df_season['dq_severity'] = df_season['dq'].apply(lambda x: 3 if x > 0 else 0)

# Efficiency (PER Proxy) - Calculated on RAW data to enable career averaging
df_season['missed_fg'] = df_season['fgAttempted'] - df_season['fgMade']
df_season['efficiency'] = (
    df_season['points'] + df_season['rebounds'] + df_season['assists'] + 
    df_season['steals'] + df_season['blocks']
) - (df_season['missed_fg'] + df_season['turnovers'] + df_season['PF'])

# 3. THE FIX: Cumulative Career Fallbacks
# We calculate career averages to fill gaps if 'prev_year' is empty (Zero Feature Trap)
df_season.sort_values(['playerID', 'year'], inplace=True)
df_season['career_discipline'] = df_season.groupby('playerID')['discipline_ratio'].expanding().mean().reset_index(level=0, drop=True)
df_season['career_efficiency'] = df_season.groupby('playerID')['efficiency'].expanding().mean().reset_index(level=0, drop=True)

# 4. Create "Previous Season" Lagged Features
df_lagged = df_season.copy()
df_lagged['year'] = df_lagged['year'] + 1  # Shift forward (Year 10 stats become Year 11 features)

# Rename to indicate history
cols_to_lag = agg_cols + ['discipline_ratio', 'dq_severity', 'efficiency', 'career_discipline', 'career_efficiency']
lag_cols = {col: f'prev_{col}' for col in cols_to_lag}
df_lagged.rename(columns=lag_cols, inplace=True)

# Merge back to the master list of players available in the Target Year
master_df = df_season[['playerID', 'year']].drop_duplicates().copy()
master_df = master_df.merge(df_lagged, on=['playerID', 'year'], how='left')

# 5. THE FIX: Apply Fallbacks
# If prev_efficiency is NaN (missed last season), use career average instead of 0
master_df['prev_efficiency'] = master_df['prev_efficiency'].fillna(master_df['prev_career_efficiency'])
master_df['prev_discipline_ratio'] = master_df['prev_discipline_ratio'].fillna(master_df['prev_career_discipline'])

# Fill remaining NaNs (True Rookies) with 0
master_df.fillna(0, inplace=True)

print("Integrity & Performance features calculated with Career Fallbacks.")
display(master_df[['playerID', 'year', 'prev_discipline_ratio', 'prev_efficiency']].tail())

### 3. Feature Engineering: Reputation & Award History
Awards are rarely given in a vacuum; reputation matters. Voters often favor established veterans or previous winners.

We engineer specific "Status" features:
* **`years_in_league`**: Seniority often correlates with respect.
* **`cumulative_all_stars`**: A proxy for "Star Power." Voters are more likely to notice a polite All-Star than a polite role player.
* **`prev_sp_wins`**: History of winning the Sportsmanship award (Dynasty effect).

**Target Definition: Graded Relevance**
We treat this as a **Learning to Rank (LTR)** problem with a 3-tier target:
* **3 (Winner)**: The player who actually won the award.
* **1 (All-Star)**: A proxy for "Nominee" or "Relevant Player." This helps the model learn to distinguish "Good Players" from "Winners," rather than just "Random Players" vs. "Winners."
* **0 (None)**: The rest of the league.

In [ ]:
# --- Feature Engineering: Reputation & Targets ---

# 1. Years in League (Veterancy Bias)
df_season.sort_values(['playerID', 'year'], inplace=True)
df_season['years_in_league'] = df_season.groupby('playerID').cumcount() + 1

# 2. Cumulative Awards (Dynasty Effect)
# Did they win Sportsmanship before?
sp_winners = awards[awards['award'].str.contains('Sportsmanship')].copy()
sp_winners['is_sp_winner'] = 1

cum_sp = sp_winners.groupby(['playerID', 'year'])['is_sp_winner'].sum().groupby('playerID').cumsum().reset_index()
cum_sp['year'] += 1 
cum_sp.rename(columns={'is_sp_winner': 'prev_sp_wins'}, inplace=True)

# All-Star Selections (Visibility)
all_stars = awards[awards['award'].str.contains('All-Star')].copy()
all_stars['is_all_star'] = 1
cum_as = all_stars.groupby(['playerID', 'year'])['is_all_star'].sum().groupby('playerID').cumsum().reset_index()
cum_as['year'] += 1
cum_as.rename(columns={'is_all_star': 'cumulative_all_stars'}, inplace=True)

# 3. Merge All Features
final_df = master_df.merge(df_season[['playerID', 'year', 'years_in_league']], on=['playerID', 'year'], how='left')
final_df = final_df.merge(cum_sp, on=['playerID', 'year'], how='left')
final_df = final_df.merge(cum_as, on=['playerID', 'year'], how='left')
final_df.fillna(0, inplace=True)

# 4. THE FIX: Graded Relevance Target
# Winner = 3
target_awards = awards[awards['award'].str.contains('Sportsmanship')][['playerID', 'year']].copy()
target_awards['is_winner'] = 3 

# All-Star = 1 (Proxy for "Nominee/Good Player")
as_awards = awards[awards['award'].str.contains('All-Star')][['playerID', 'year']].copy()
as_awards['relevance_as'] = 1

# Merge Targets
final_df = final_df.merge(target_awards, on=['playerID', 'year'], how='left')
final_df = final_df.merge(as_awards, on=['playerID', 'year'], how='left')

# Combine: Winner (3) overrides All-Star (1), others are 0
final_df['is_winner'] = final_df['is_winner'].fillna(final_df['relevance_as']).fillna(0)
final_df.drop(columns=['relevance_as'], inplace=True)

# Filter: Remove Rookies (Year 1)
final_df = final_df[final_df['years_in_league'] > 1].copy()

print(f"Final Dataset Shape: {final_df.shape}")
print("Target Distribution (0=None, 1=AllStar, 3=Winner):")
print(final_df['is_winner'].value_counts())
display(final_df.head())

### 4. Modeling Strategy: SOTA Ensemble
Predicting a single winner out of hundreds of players is highly volatile. To stabilize predictions, we employ an ensemble of three distinct architectures:

1.  **XGBoost Ranker (`rank:ndcg`)**:
    * Specializes in optimizing the list order directly.
    * Tuned with high regularization to prevent overfitting to historical outliers.

2.  **LightGBM Ranker (`lambdarank`)**:
    * Uses a different gradient boosting implementation that often captures different signal patterns.
    * Explicitly defined label gains (0, 1, 10) to heavily prioritize finding the "Winner."

3.  **Random Forest Regressor**:
    * Acts as a stabilizer. While Rankers care about *relative* order, the Random Forest looks at *absolute* feature strength. It prevents the rankers from hallucinating a winner in a weak year.

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# --- 1. Model Definitions (SOTA Ensemble) ---

# A. XGBoost Ranker (The Specialist)
# Tuned for small data: deeper trees but high regularization
xgb_model = xgb.XGBRanker(
    objective='rank:ndcg',
    eval_metric='ndcg@3',
    learning_rate=0.03,    # Slower learning for better generalization
    n_estimators=1000,
    max_depth=5,
    min_child_weight=2,    # Prevents overfitting to single outliers
    lambdarank_pair_method='topk',
    random_state=42
)

# B. LightGBM Ranker (The Speedster)
# FIXED: Label gain must match the max label index (0, 1, 3 -> size 4)
lgb_model = lgb.LGBMRanker(
    objective='lambdarank',
    metric='ndcg',
    learning_rate=0.03,
    n_estimators=1000,
    max_depth=4,
    label_gain=[0, 1, 0, 10], # Index 0=0, 1=1, 2=0 (unused), 3=10 (Winner)
    random_state=42,
    verbose=-1
)

# C. Random Forest Regressor (The Stabilizer)
# Treats ranking as a regression problem (predicting relevance score 0, 1, 3)
# Good for small datasets where LTR overfits.
rf_model = RandomForestRegressor(
    n_estimators=500,
    max_depth=6,           # Constrained depth
    max_features='sqrt',   # Forces diversity
    random_state=42
)

# --- 2. Validation Loop with Ensemble ---

features = [
    'prev_discipline_ratio', 
    'prev_dq_severity', 
    'prev_PF', 
    'prev_efficiency', 
    'prev_minutes', 
    'years_in_league', 
    'cumulative_all_stars', 
    'prev_sp_wins'
]

all_results = []
print(f"{'='*20} STARTING ENSEMBLE VALIDATION {'='*20}")

for test_year in range(4, 11):
    print(f"\nProcessing Year {test_year}...")
    
    # 1. Split Data
    train_df = final_df[final_df['year'] < test_year].copy().sort_values('year')
    test_df = final_df[final_df['year'] == test_year].copy().sort_values('year')
    
    if test_df['is_winner'].max() < 3: continue

    train_df = train_df[train_df['prev_minutes'] > 200]
    test_df = test_df[test_df['prev_minutes'] > 200]
    
    # Check if winner survived filter (Safety)
    if test_df['is_winner'].max() < 3:
        print("   ! Winner was filtered out by low minutes. Skipping.")
        continue

    # Prepare Inputs
    X_train, y_train = train_df[features], train_df['is_winner']
    qid_train = train_df['year']
    
    X_test, y_test = test_df[features], test_df['is_winner']
    qid_test = test_df['year']
    
    # Group counts for LGBM
    group_train = train_df.groupby('year').size().to_list()
    group_test = test_df.groupby('year').size().to_list()

    # --- 3. Train Individual Models ---
    
    # Fit XGB
    xgb_model.fit(X_train, y_train, qid=qid_train, verbose=False)
    pred_xgb = xgb_model.predict(X_test)
    
    # Fit LGBM
    lgb_model.fit(X_train, y_train, group=group_train, eval_group=[group_test], eval_set=[(X_test, y_test)])
    pred_lgb = lgb_model.predict(X_test)
    
    # Fit RF (Pointwise)
    rf_model.fit(X_train, y_train)
    pred_rf = rf_model.predict(X_test)

    # --- 4. Ensemble Voting (Min-Max Scaling + Averaging) ---
    # We must scale scores because XGB might output 0-10 and RF 0-3
    scaler = MinMaxScaler()
    
    p_xgb_norm = scaler.fit_transform(pred_xgb.reshape(-1, 1)).flatten()
    p_lgb_norm = scaler.fit_transform(pred_lgb.reshape(-1, 1)).flatten()
    p_rf_norm  = scaler.fit_transform(pred_rf.reshape(-1, 1)).flatten()
    
    # Weighted Average: Give more weight to Stability (RF) if Rankers are failing
    # Weights: XGB(35%), LGB(35%), RF(30%)
    final_score = (p_xgb_norm * 0.35) + (p_lgb_norm * 0.35) + (p_rf_norm * 0.30)
    
    # --- 5. Evaluation ---
    temp_res = test_df.copy()
    temp_res['score'] = final_score
    
    ranked = temp_res.sort_values('score', ascending=False).reset_index(drop=True)
    
    try:
        winner_rank = ranked[ranked['is_winner'] == 3].index[0] + 1
        winner_id = ranked[ranked['is_winner'] == 3]['playerID'].values[0]
    except IndexError:
        winner_rank = 99
        winner_id = "Unknown"
        
    print(f"   🏆 Ensemble Rank: {winner_rank} ({winner_id})")
    
    all_results.append({
        'Year': test_year,
        'Rank': winner_rank,
        'MRR': 1/winner_rank
    })

# --- Summary ---
if all_results:
    res_df = pd.DataFrame(all_results)
    print("\n" + "="*30)
    print(f"Final Avg Rank: {res_df['Rank'].mean():.2f}")
    print(f"Final MRR: {res_df['MRR'].mean():.4f}")

### 5. Validation Loop (Walk-Forward)
We validate the ensemble by simulating the voting process for Years 4 through 10.
* **Metric**: **Rank of True Winner**. (Did we put the actual winner in the #1 spot?)
* **Ensemble Logic**: We normalize the scores from all three models (MinMax scaling) and compute a weighted average:
    * 35% XGBoost
    * 35% LightGBM
    * 30% Random Forest

In [ ]:
# --- 4. Summary & Comparison ---

if all_results:
    results_df = pd.DataFrame(all_results)

    # Since we are using a single Ensemble approach, we summarize the overall performance
    # rather than grouping by 'Model'.
    summary = {
        'Avg Winner Rank': results_df['Rank'].mean(),
        'Mean Reciprocal Rank (MRR)': results_df['MRR'].mean()
    }

    print("\n🏆 Ensemble Performance Summary 🏆")
    print(f"   Avg Winner Rank: {summary['Avg Winner Rank']:.2f}")
    print(f"   Mean Reciprocal Rank: {summary['Mean Reciprocal Rank (MRR)']:.4f}")

    # Detailed Year-by-Year
    print(f"\n--- Year-by-Year Ensemble Performance ---")
    display(results_df[['Year', 'Rank', 'MRR']])
else:
    print("No results generated.")

### 6. Final Prediction for Test Season (Year 11)
We now retrain the entire ensemble on the full 10-year history and generate the "Ballot" for Year 11.


The output below represents the **Top 5 Candidates** most likely to win the Sportsmanship Award based on their behavior in Year 10 and their career reputation.

In [ ]:
# --- 5. Final Prediction for Year 11 (Ensemble) ---

print(f"{'='*30} YEAR 11 PREDICTIONS {'='*30}")

# 1. Prepare Training Data (All Years 1-10)
# Sort by year is crucial for LTR models
train_final = final_df[final_df['year'] < 11].copy().sort_values('year')
test_final = final_df[final_df['year'] == 11].copy().sort_values('year')

# Apply Elite Filter to Training (Reduce noise)
train_final = train_final[train_final['prev_minutes'] > 200]

# Prepare Feature Matrices
X_train = train_final[features]
y_train = train_final['is_winner']
qid_train = train_final['year']
group_train = train_final.groupby('year').size().to_list()

X_test = test_final[features]

# 2. Train Individual Models on Full History

# A. XGBoost
xgb_model.fit(X_train, y_train, qid=qid_train, verbose=False)
pred_xgb = xgb_model.predict(X_test)

# B. LightGBM (LGBM does not use evaluation sets here, just trains)
lgb_model.fit(X_train, y_train, group=group_train)
pred_lgb = lgb_model.predict(X_test)

# C. Random Forest
rf_model.fit(X_train, y_train)
pred_rf = rf_model.predict(X_test)

# 3. Ensemble Voting (Min-Max Scaling + Weighted Average)
scaler = MinMaxScaler()

p_xgb_norm = scaler.fit_transform(pred_xgb.reshape(-1, 1)).flatten()
p_lgb_norm = scaler.fit_transform(pred_lgb.reshape(-1, 1)).flatten()
p_rf_norm  = scaler.fit_transform(pred_rf.reshape(-1, 1)).flatten()

# Same weights as validation: XGB(35%), LGB(35%), RF(30%)
final_scores = (p_xgb_norm * 0.35) + (p_lgb_norm * 0.35) + (p_rf_norm * 0.30)

# 4. Output Top Candidates
test_final['pred_score'] = final_scores
top_picks = test_final.sort_values('pred_score', ascending=False).head(5)

print("\n🏆 Top 5 Candidates for Year 11:")
for idx, row in top_picks.iterrows():
    print(f"   {idx+1}. {row['playerID']} (Score: {row['pred_score']:.4f})")
    print(f"       > Prev Disc: {row['prev_discipline_ratio']:.1f} | Prev Eff: {row['prev_efficiency']:.0f} | Prev Wins: {row['prev_sp_wins']}")